In [21]:
import module.DataPreprocessing as data_pp
import numpy as np

# 클래스 수정 시 import 리로드
import importlib
importlib.reload(data_pp)

# 파일경로, 라벨 정보 추출
root_folder = "..\\01.Data\\01.fish"
dp = data_pp.DataPreprocessing()
file_path, label = dp.get_file_path(root_folder, "png")
len(file_path), len(label)

(9281, 9281)

In [22]:
# 라벨 인덱싱
indexed_label_arr = dp.label_indexing(label)
dp.get_label_dic()

{'Black Sea Sprat': 0,
 'Gilt-Head Bream': 1,
 'Hourse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [23]:
# 데이터셋 나누기
test_set, valid_set, train_set = dp.split_test_vaild_train(np.array(file_path), indexed_label_arr, 0.15)
len(test_set), len(valid_set), len(train_set)      

(1403, 1403, 6475)

In [24]:
# 배치사이즈로 나누기
test_X, test_Y = dp.make_batch_arr(test_set, 10)
valid_X, valid_Y = dp.make_batch_arr(valid_set, 10)
train_X, train_Y = dp.make_batch_arr(train_set, 10)

In [42]:
# 폴더명, 라벨 정보 일치 여부 확인
print(dp.verify_dataset(test_X, test_Y))
print(dp.verify_dataset(valid_X, valid_Y))
print(dp.verify_dataset(train_X, train_Y))

True
True
True


In [26]:
# 배치 갯수
len(train_X), len(train_Y), len(valid_X), len(valid_Y), len(test_X), len(test_Y)

(647, 647, 140, 140, 140, 140)

In [27]:
import module.CustomDataset as cd
importlib.reload(cd)
train_X_1d = [path for path_lst in train_X for path in path_lst]
train_Y_1d = [label for label_lst in train_Y for label in label_lst]
trainset = cd.CustomDataset(train_X_1d, train_Y_1d)

valid_X_1d = [path for path_lst in valid_X for path in path_lst]
valid_Y_1d = [label for label_lst in valid_Y for label in label_lst]
validset = cd.CustomDataset(valid_X_1d, valid_Y_1d)


In [28]:
len(trainset), len(valid_set)

(6470, 1403)

In [29]:
import module.CNN as CNN
import module.RunCNN as RunCNN

# 클래스 수정 시 import 리로드
importlib.reload(CNN)
importlib.reload(RunCNN)

learning_rate = 0.001
training_epochs = 10
batch_size = 10

runner = RunCNN.RunCNN()

train_data_loader = runner.get_dataloader(trainset, batch_size, True, True)
valid_data_loader = runner.get_dataloader(validset, batch_size, False, True)

device = runner.get_device()
model = CNN.CNN().to(device)

In [30]:
len(train_data_loader), len(valid_data_loader)

(647, 140)

In [ ]:
runner.run_epoch(training_epochs, train_data_loader, valid_data_loader, device, model, learning_rate)